### Building a machine learning model to predict player's position and his rating

In [8]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [9]:
import zipfile
with zipfile.ZipFile("archive.zip") as z:
    with z.open("male_players.csv") as f:
        data = pd.read_csv(f)

data.head()

C:\Users\pranj\AppData\Local\Temp\ipykernel_21436\1152988356.py:4: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f)


,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_id,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,181500000.0,230000.0,24,1998-12-20,182,75,73.0,Paris Saint Germain,16.0,Ligue 1,1.0,LW,7.0,NaN,2018-07-01,2024.0,18,France,1335.0,LW,10.0,Right,4,5,5,High/Low,Unique,Yes,349400000.0,"#Speedster, #Dribbler, #Acrobat, #Clinical fin...","Quick Step +, Rapid, Flair, Trivela",97.0,90.0,80.0,92.0,36.0,78.0,78,94,73,86,84,93,80,69,71,92,97,97,93,93,82,90,88,88,77,83,64,38,93,83,84,88.0,26,34,32,13,5,7,11,6,NaN,90+3,90+3,90+3,91,91,91,91,91,89+3,89+3,89+3,89+3,81+3,81+3,81+3,89+3,68+3,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,185000000.0,340000.0,22,2000-07-21,195,94,10.0,Manchester City,13.0,Premier League,1.0,ST,9.0,NaN,2022-07-01,2027.0,36,Norway,1352.0,ST,9.0,Left,3,3,5,High/Medium,Unique,Yes,356100000.0,"#Aerial threat, #Distance shooter, #Strength, ...","Acrobatic +, Power Header, Quick Step",89.0,93.0,66.0,80.0,45.0,88.0,47,96,83,77,90,79,77,62,53,82,82,94,76,94,72,94,93,76,93,86,87,43,96,74,84,87.0,38,47,29,7,14,13,11,7,NaN,90+3,90+3,90+3,82,86,86,86,82,82+3,82+3,82+3,79+3,74+3,74+3,74+3,79+3,62+3,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,103000000.0,350000.0,32,1991-06-28,181,75,10.0,Manchester City,13.0,Premier League,1.0,SUB,17.0,NaN,2015-08-30,2025.0,7,Belgium,1325.0,CAM,7.0,Right,5,4,5,High/Medium,Unique,Yes,190600000.0,"#Dribbler, #Playmaker, #Distance shooter, #Cro...","Pinged Pass +, Dead Ball, Incisive Pass, Long ...",72.0,88.0,94.0,87.0,65.0,78.0,95,85,55,94,83,86,92,83,94,92,72,72,74,92,78,92,72,88,74,92,75,66,88,95,83,88.0,66,70,53,15,13,5,10,13,NaN,83+3,83+3,83+3,87,88,88,88,87,89+2,89+2,89+2,88+3,90+1,90+1,90+1,88+3,79+3,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,/player/158023/lionel-messi/240002,24.0,2.0,2023-09-22,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,41000000.0,23000.0,36,1987-06-24,169,67,112893.0,Inter Miami,39.0,Major League Soccer,1.0,RF,10.0,NaN,2023-07-16,2025.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Low/Low,Unique,Yes,61500000.0,"#Dribbler, #Playmaker, #FK Specialist, #Acroba...","Technical +, Finesse Shot, Dead Ball, Pinged P...",80.0,87.0,90.0,94.0,33.0,64.0,83,89,60,91,86,96,93,93,90,93,87,74,91,88,95,83,71,70,68,90,44,40,91,92,75,96.0,20,35,24,6,11,15,14,8,NaN,85+3,85+3,85+3,90,89,89,89,90,91-1,91-1,91-1,89+1,85+3,85+3,85+3,89+1,64+3,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180021 entries, 0 to 180020
Columns: 109 entries, player_id to gk
dtypes: float64(20), int64(43), object(46)
memory usage: 149.7+ MB


In [11]:
data = data.drop(['player_id','player_url','fifa_version','fifa_update','update_as_of','short_name','long_name','dob','club_team_id',
                  'league_id','club_loaned_from','club_joined_date','club_contract_valid_until_year','nationality_id','nation_team_id',
                 'player_tags','player_traits','value_eur','wage_eur','height_cm','weight_kg','club_name','league_name','club_jersey_number','nationality_name',
           'nation_jersey_number','preferred_foot','body_type','real_face','release_clause_eur','potential','club_position','nation_position'], axis = 1)

### Data preprocessing

In [12]:
cols_to_modify = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']

In [13]:
def add_values(value):
    if '+' in str(value):
        parts = str(value).split('+')
        return int(parts[0]) + int(parts[1])
    elif '-' in str(value):
        parts = str(value).split('-')
        return int(parts[0]) - int(parts[1])
    else:
        return int(value)
#data[[col+'_org', col+'_grow']] = data[col].str.split('+', expand = True)

In [14]:
for col in cols_to_modify:
    data[col] = data[col].apply(add_values)

In [15]:
def replace_nan(data):
    for column in data.columns:
        if pd.api.types.is_numeric_dtype(data[column]):
            data[column].fillna(0, inplace=True)
        else:
            data[column].fillna('Not Applicable', inplace=True)

In [16]:
replace_nan(data)

In [17]:
data[['work_rate_attack','work_rate_defence']] = data['work_rate'].str.split('/', expand=True)

In [18]:
data.drop('work_rate', axis = 1, inplace = True)

In [19]:
def player_position_trim(value):
    trim_pos = str(value).split(',')
    return trim_pos[0]

In [20]:
data['player_positions'] = data['player_positions'].apply(player_position_trim)

In [21]:
conditions = [
    (data['player_positions'] == 'CF') | (data['player_positions'] == 'LW') | (data['player_positions'] == 'RW') | (data['player_positions'] == 'ST'),
    (data['player_positions'] == 'RM') | (data['player_positions'] == 'LM') | (data['player_positions'] == 'CM') | (data['player_positions'] == 'CAM') | (data['player_positions'] == 'CDM'),
    (data['player_positions'] == 'CB') | (data['player_positions'] == 'RB') | (data['player_positions'] == 'LB') | (data['player_positions'] == 'RWB') | (data['player_positions'] == 'LWB'),
    (data['player_positions'] == 'GK')]

values = ['Forward', 'Midfielder', 'Defender', 'Goalkeeper']

data['position_4'] = np.select(conditions, values, default='Other')

In [22]:
data.head()

,player_positions,overall,age,league_level,weak_foot,skill_moves,international_reputation,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,work_rate_attack,work_rate_defence,position_4
0,ST,91,24,1.0,4,5,5,97.0,90.0,80.0,92.0,36.0,78.0,78,94,73,86,84,93,80,69,71,92,97,97,93,93,82,90,88,88,77,83,64,38,93,83,84,88.0,26,34,32,13,5,7,11,6,0.0,93,93,93,91,91,91,91,91,92,92,92,92,84,84,84,92,71,66,66,66,71,66,57,57,57,66,21,High,Low,Forward
1,ST,91,22,1.0,3,3,5,89.0,93.0,66.0,80.0,45.0,88.0,47,96,83,77,90,79,77,62,53,82,82,94,76,94,72,94,93,76,93,86,87,43,96,74,84,87.0,38,47,29,7,14,13,11,7,0.0,93,93,93,82,86,86,86,82,85,85,85,82,77,77,77,82,65,66,66,66,65,63,65,65,65,63,22,High,Medium,Forward
2,CM,91,32,1.0,5,4,5,72.0,88.0,94.0,87.0,65.0,78.0,95,85,55,94,83,86,92,83,94,92,72,72,74,92,78,92,72,88,74,92,75,66,88,95,83,88.0,66,70,53,15,13,5,10,13,0.0,86,86,86,87,88,88,88,87,91,91,91,91,91,91,91,91,82,83,83,83,82,78,73,73,73,78,24,High,Medium,Midfielder
3,CF,90,36,1.0,4,4,5,80.0,87.0,90.0,94.0,33.0,64.0,83,89,60,91,86,96,93,93,90,93,87,74,91,88,95,83,71,70,68,90,44,40,91,92,75,96.0,20,35,24,6,11,15,14,8,0.0,88,88,88,90,89,89,89,90,90,90,90,90,88,88,88,90,67,66,66,66,67,62,52,52,52,62,22,Low,Low,Forward
4,CF,90,35,1.0,4,4,5,79.0,88.0,83.0,87.0,39.0,78.0,75,91,90,89,88,87,82,73,76,91,78,79,77,92,72,87,85,82,82,81,63,39,92,90,85,90.0,43,24,18,13,11,5,5,7,0.0,90,90,90,86,89,89,89,86,90,90,90,89,85,85,85,89,67,67,67,67,67,63,58,58,58,63,21,Medium,Medium,Forward


In [23]:
data_num = data.drop(['player_positions','overall','league_level','weak_foot','skill_moves','international_reputation','work_rate_attack',
                     'work_rate_defence','position_4'], axis = 1)
data_ord = data[['league_level','weak_foot','skill_moves','international_reputation',]]

data_cat = data[['work_rate_attack','work_rate_defence']]

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in data_cat:
    data_cat[col] = pd.DataFrame(le.fit_transform(data_cat[col]), columns = [col])

C:\Users\pranj\AppData\Local\Temp\ipykernel_21436\1900511266.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cat[col] = pd.DataFrame(le.fit_transform(data_cat[col]), columns = [col])
C:\Users\pranj\AppData\Local\Temp\ipykernel_21436\1900511266.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cat[col] = pd.DataFrame(le.fit_transform(data_cat[col]), columns = [col])


In [25]:
X = pd.concat([data_num, data_ord, data_cat], axis = 1)

In [26]:
Y = data['position_4']

In [27]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_std = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_std,Y,test_size=0.3,random_state=42)

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,multilabel_confusion_matrix
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X_train,Y_train)
Y_pred_4 = rfc.predict(X_test)

In [30]:
accuracy_score(Y_test,Y_pred_4)

0.9207695298757569

In [31]:
precision_score(Y_test,Y_pred_4, average = None)

array([0.95040252, 0.89237392, 1.        , 0.88472679])

In [32]:
recall_score(Y_test,Y_pred_4, average = None)

array([0.9588232 , 0.84018529, 1.        , 0.90471409])

In [33]:
multilabel_confusion_matrix(Y_test, Y_pred_4)

array([[[35519,   881],
        [  725, 16882]],

       [[42595,  1050],
        [ 1656,  8706]],

       [[47888,     0],
        [    0,  6119]],

       [[31740,  2348],
        [ 1898, 18021]]], dtype=int64)

In [34]:
Y = data['player_positions']

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_std,Y,test_size=0.3,random_state=78)

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,multilabel_confusion_matrix
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X_train,Y_train)
Y_pred = rfc.predict(X_test)

In [37]:
accuracy_score(Y_test,Y_pred)

0.767159812616883

In [38]:
precision_score(Y_test,Y_pred, average = None)

array([0.63857466, 0.91179913, 0.7497657 , 0.75581395, 0.69972596,
       0.99983496, 0.64938212, 0.49140893, 0.52262443, 0.48051948,
       0.6761242 , 0.50319189, 0.58634538, 0.5       , 0.86696544])

In [39]:
recall_score(Y_test,Y_pred, average = None)

array([0.66470415, 0.93334785, 0.72644722, 0.17857143, 0.85219326,
       1.        , 0.70656472, 0.48310811, 0.21076642, 0.11044776,
       0.63820111, 0.459849  , 0.23453815, 0.10062893, 0.95591678])

In [40]:
multilabel_confusion_matrix(Y_test, Y_pred)

array([[[49332,  1278],
        [ 1139,  2258]],

       [[43996,   829],
        [  612,  8570]],

       [[48534,  1068],
        [ 1205,  3200]],

       [[53622,    21],
        [  299,    65]],

       [[45414,  2301],
        [  930,  5362]],

       [[47948,     1],
        [    0,  6058]],

       [[48767,  1447],
        [ 1113,  2680]],

       [[49567,  1480],
        [ 1530,  1430]],

       [[52700,   211],
        [  865,   231]],

       [[53632,    40],
        [  298,    37]],

       [[48839,  1210],
        [ 1432,  2526]],

       [[49770,  1323],
        [ 1574,  1340]],

       [[52556,   206],
        [  953,   292]],

       [[53657,    32],
        [  286,    32]],

       [[45189,  1128],
        [  339,  7351]]], dtype=int64)

In [43]:
Y = data['overall']
Y.head()

0    91
1    91
2    91
3    90
4    90
Name: overall, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_std,Y,test_size=0.3,random_state=42)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
regressor = RandomForestRegressor(n_estimators=50, random_state=56)
regressor.fit(X_train, Y_train)
predictions = regressor.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(Y_test, predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 0.49023780621030605
R-squared: 0.9900884320681439
